In [1]:
import numpy as np
import pandas as pd

In [95]:
def reformat(infile, outfile):
    raw = pd.read_csv(infile, delimiter='\t', names=["1gram", "year", "occ"], 
                      dtype={"1gram":str, "year":np.uint16, "occ":np.uint32}, engine='c', quoting=3, encoding='latin-1')
    years = np.array(raw["year"])
    counts = np.array(raw["occ"])

    # break up each word into its own sub-array
    shifted = np.hstack((raw["1gram"][1:], ["this has a space in it so will match nothing"]))
    inds = np.where(raw["1gram"] != shifted)[0] + 1

    years_each_word = np.split(years, inds)
    counts_each_word = np.split(counts, inds)
    
    # fill in zeroes on skipped years
    min_year = np.min(years)
    max_year = np.max(years)

    full_years = np.arange(min_year, max_year+1)

    # axes (word, year)
    full_counts_each_word = np.zeros((len(years_each_word), full_years.size), dtype=np.int32)

    # not ideal, but only have to run this once...
    # possibly do this whole thing in C instead
    for i in range(len(years_each_word)):
        for j in range(len(years_each_word[i])):
            ind = np.where(years_each_word[i][j] == full_years)[0][0]
            full_counts_each_word[i, ind] += counts_each_word[i][j]
            
    with open(outfile, 'ba') as f:
        np.savetxt(f, full_counts_each_word, fmt='%u', delimiter=',')

In [96]:
outfile = "../ngram_data/full_counts.csv"
for i in range(10):
    name = f"../ngram_data/googlebooks-eng-1M-1gram-20090715-occonly{i}.csv"
    reformat(name, outfile)
    print(i)

0
1
2
3
4
5
6
7
8
9


In [55]:
name = f"../ngram_data/googlebooks-eng-1M-1gram-20090715-occonly0.csv"
raw = pd.read_csv(name, delimiter='\t', names=["1gram", "year", "occ"], 
                      dtype={"1gram":str, "year":np.uint16, "occ":np.uint32})

In [56]:
years = np.array(raw["year"])
counts = np.array(raw["occ"])

# break up each word into its own sub-array
shifted = np.hstack((raw["1gram"][1:], ["this has a space in it so will match nothing"]))
inds = np.where(raw["1gram"] != shifted)[0] + 1

years_each_word = np.split(years, inds)
counts_each_word = np.split(counts, inds)

In [77]:
# fill in zeroes on skipped years
min_year = np.min(years)
max_year = np.max(years)

full_years = np.arange(min_year, max_year+1)

# axes (word, year)
full_counts_each_word = np.zeros((len(years_each_word), full_years.size), dtype=np.int32)

for i in range(len(years_each_word)):
    for j in range(len(years_each_word[i])):
        ind = np.where(years_each_word[i][j] == full_years)[0][0]
        full_counts_each_word[i, ind] += counts_each_word[i][j]

In [86]:
# write to file in same order as read

outfile = "../ngram_data/test.csv"

with open(outfile, 'ba') as f:
    np.savetxt(f, full_counts_each_word, fmt='%u', delimiter=',')